In [3]:
import sys
import importlib

# Force reload of the module
if 'portfolio_optimizer' in sys.modules:
    del sys.modules['portfolio_optimizer']

sys.path.insert(0, '../src')

In [4]:
import sys
sys.path.append('../src')

from portfolio_optimizer import PortfolioOptimizer
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("Imports successful!")

Imports successful!


In [7]:
with open('../src/portfolio_optimizer.py', 'r') as f:
    lines = f.readlines()
    # Print lines 24-45 (around fetch_data method)
    for i, line in enumerate(lines[23:45], start=24):
        print(f"{i}: {line}", end='')

24:         """Download historical price data from Yahoo Finance"""
25:         print(f"Fetching data for {len(self.tickers)} tickers...")
26:         
27:         # Download with auto_adjust=True to get adjusted prices directly
28:         data = yf.download(
29:             self.tickers, 
30:             start=self.start_date, 
31:             end=self.end_date,
32:             auto_adjust=True,
33:             progress=False
34:         )
35:         
36:         # If single ticker, data is already a DataFrame with OHLCV columns
37:         if len(self.tickers) == 1:
38:             self.prices = data['Close'].to_frame()
39:             self.prices.columns = self.tickers
40:         else:
41:             # Multiple tickers - extract Close prices
42:             self.prices = data['Close']
43:         
44:         # Drop any missing values
45:         self.prices = self.prices.dropna()


In [8]:
# Stock selection - diversified across sectors
# Tech: AAPL, MSFT
# Finance: JPM, GS
# Healthcare: JNJ, UNH
# Consumer: PG, KO
# Energy: XOM
# Communication: GOOGL

tickers = ['AAPL', 'MSFT', 'JPM', 'GS', 'JNJ', 'UNH', 'PG', 'KO', 'XOM', 'GOOGL']

# Use 3 years of historical data
start_date = '2022-01-01'
end_date = '2025-01-01'

print(f"Portfolio: {', '.join(tickers)}")
print(f"Period: {start_date} to {end_date}")

Portfolio: AAPL, MSFT, JPM, GS, JNJ, UNH, PG, KO, XOM, GOOGL
Period: 2022-01-01 to 2025-01-01


In [9]:
portfolio = PortfolioOptimizer(tickers, start_date, end_date)

prices = portfolio.fetch_data()
returns = portfolio.calculate_returns()

print(f"\nData shape: {prices.shape}")
print(f"\nPrice statistics:")
print(prices.describe())

Fetching data for 10 tickers...
Downloaded 753 days of data

Data shape: (753, 10)

Price statistics:
Ticker        AAPL       GOOGL          GS         JNJ         JPM  \
count   753.000000  753.000000  753.000000  753.000000  753.000000   
mean    176.192723  131.586113  359.092723  150.771219  150.403479   
std      29.508373   27.240840   80.887767    6.785908   38.224538   
min     123.161949   82.804367  255.862808  135.852158   94.269768   
25%     153.548462  109.512550  303.488922  146.213638  121.594955   
50%     171.348129  130.543625  324.525726  150.641006  136.982315   
75%     189.767822  150.731110  390.201233  155.927521  184.759796   
max     257.853760  195.892807  593.458618  166.509323  244.000839   

Ticker          KO        MSFT          PG         UNH         XOM  
count   753.000000  753.000000  753.000000  753.000000  753.000000  
mean     57.348757  328.513876  143.707140  490.687662   96.236885  
std       4.031220   71.086264   12.762398   37.460196   13.

In [11]:
equal_weight = portfolio.equal_weight_portfolio()

max_sharpe = portfolio.optimize_sharpe()

min_vol = portfolio.optimize_min_volatility()

print("=" * 60)
print("PORTFOLIO COMPARISON")
print("=" * 60)

strategies = {
    'Equal Weight': equal_weight,
    'Max Sharpe Ratio': max_sharpe,
    'Min Volatility': min_vol
}

for name, stats in strategies.items():
    print(f"\n{name}:")
    print(f"  Expected Return: {stats['return']*100:.2f}%")
    print(f"  Volatility: {stats['volatility']*100:.2f}%")
    print(f"  Sharpe Ratio: {stats['sharpe']:.3f}")

PORTFOLIO COMPARISON

Equal Weight:
  Expected Return: 11.84%
  Volatility: 14.69%
  Sharpe Ratio: 0.806

Max Sharpe Ratio:
  Expected Return: 19.31%
  Volatility: 19.43%
  Sharpe Ratio: 0.994

Min Volatility:
  Expected Return: 5.49%
  Volatility: 12.32%
  Sharpe Ratio: 0.445


In [17]:
import plotly.graph_objects as go 

fig = go.Figure()

for strategy_name, stats in strategies.items():
    weights = stats['weights']
    fig.add_trace(go.Bar(
        name = strategy_name,
        x = list(weights.keys()),
        y = [weights[ticker] * 100 for ticker in weights.keys()]
    ))

fig.update_layout(
    title = 'Portfolio Allocation Comparison',
    xaxis_title = 'Ticker',
    yaxis_title = 'Weight (%)',
    barmode = 'group',
    height = 500,
    template = 'plotly_white'
)

#fig.show()

fig.write_html('../visualizations/weight_comparison.html')

In [19]:
target_returns, frontier_volatility = portfolio.calculate_efficient_frontier(n_points = 100)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = frontier_volatility, 
    y = target_returns,
    mode = 'lines',
    name = 'Efficient Frontier',
    line = dict(color = 'blue', width = 3)
))

colors = {'Equal Weight': 'gray', 'Max Sharpe Ratio': 'green', 'Min Volatility': 'red'}
for name, stats in strategies.items():
    fig.add_trace(go.Scatter(
        x = [stats['volatility']],
        y = [stats['return']],
        mode = 'markers',
        name = name,
        marker = dict(size = 15, color = colors[name], symbol = 'star'),
        text = f"{name}<br>Return: {stats['return']*100:.1f}%<br>Vol: {stats['volatility']*100:.1f}%<br>Sharpe: {stats['sharpe']:.2f}",
        hoverinfo = 'text'
    ))

assets_returns = returns.mean() * 252
assets_vols = returns.std() * np.sqrt(252)

fig.add_trace(go.Scatter(
    x = assets_vols,
    y = assets_returns,
    mode = 'markers+text',
    name = 'Individual Assets',
    marker = dict(size = 10, color = 'lightblue'),
    text = tickers,
    textposition = 'top center',
    hovertemplate='%{text}<br>Return: %{y:.1%}<br>Volatility: %{x:.1%}<extra></extra>'
))

fig.update_layout(
    title='Efficient Frontier Analysis',
    xaxis_title='Volatility (Annual)',
    yaxis_title='Expected Return (Annual)',
    height=600,
    template='plotly_white',
    hovermode='closest',
    xaxis=dict(tickformat='.1%'),
    yaxis=dict(tickformat='.1%')
)

#fig.show()

fig.write_html('../visualizations/efficient_frontier.html')

In [20]:
weight_df = pd.DataFrame({
    strategy: [weights[ticker] for ticker in tickers]
    for strategy, weights in [(name, stats['weights']) for name, stats in strategies.items()]
}, index = tickers)

weight_df['Difference (MaxSharpe - EqualWeight)'] = weight_df['Max Sharpe Ratio'] - weight_df['Equal Weight']

display((weight_df * 100).round(2))

weight_df.to_csv('../data/portfolio_weights.csv')
print("\nWeights saved to data/portfolio_weights.csv")

,Equal Weight,Max Sharpe Ratio,Min Volatility,Difference (MaxSharpe - EqualWeight)
AAPL,10.0,10.28,0.00,0.28
MSFT,10.0,1.46,4.66,-8.54
JPM,10.0,30.39,2.04,20.39
GS,10.0,2.86,2.29,-7.14
JNJ,10.0,0.00,30.25,-10.00
UNH,10.0,0.00,4.66,-10.00
PG,10.0,3.86,16.21,-6.14
KO,10.0,0.00,23.16,-10.00
XOM,10.0,51.13,13.03,41.13
GOOGL,10.0,0.00,3.69,-10.00



Weights saved to data/portfolio_weights.csv


In [21]:
summary = pd.DataFrame({
    'Strategy': list(strategies.keys()),
    'Expected Return (%)': [stats['return']*100 for stats in strategies.values()],
    'Volatility (%)': [stats['volatility']*100 for stats in strategies.values()],
    'Sharpe Ratio': [stats['sharpe'] for stats in strategies.values()]
})

summary = summary.round(3)
print("\n" + "="*60)
print("FINAL PORTFOLIO SUMMARY")
print("="*60)
display(summary)

sharpe_improvement = ((max_sharpe['sharpe'] - equal_weight['sharpe']) / equal_weight['sharpe']) * 100
print(f"\nSharpe Ratio Improvement: {sharpe_improvement:.1f}%")
print(f"Risk Reduction (Min Vol vs Equal Weight): {((equal_weight['volatility'] - min_vol['volatility'])/equal_weight['volatility'])*100:.1f}%")


FINAL PORTFOLIO SUMMARY


,Strategy,Expected Return (%),Volatility (%),Sharpe Ratio
0,Equal Weight,11.843,14.694,0.806
1,Max Sharpe Ratio,19.306,19.429,0.994
2,Min Volatility,5.488,12.324,0.445



Sharpe Ratio Improvement: 23.3%
Risk Reduction (Min Vol vs Equal Weight): 16.1%
